In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\Affinity\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
#from tensorflow.keras.datasets import mnist
#(x_train, y_train),(x_test,y_test) = mnist.load_data()
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#batch_xs, bat_ys = mnist.train.next_batch(100)

#input placeholders
# 자리만 확보하고 나중에 데이터를 feed_dict형태로 넣게된다.

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
# -1,28,28,1 none으로 지정되어서 -1로 감.

Y = tf.placeholder(tf.float32, [None, 10])


#Conv L1 ImgIn shape=(?, 28, 28, 1)
# Filter W1 size=[3,3,1,6] 
# 필터의 형태. 3 by 3 글고 32 개의 필터.
W1 = tf.Variable(tf.random_normal([3,3,1, 32],stddev=0.03))

#L1
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1_img = tf.nn.relu(L1)
# activatin function= 하나의 출력으로 내놓는다.
# nvanishing gradient 를 방지. back pro에서. 
# relu= X 값이 커지는 것 을 방지. 

#Max pooling MP1 size 
MP1 = tf.nn.max_pool(L1_img, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#사이즈를 반값으로 줄여준다.


#Conv L2 ImgIn shape
#Filter W2 size
W2 = tf.Variable(tf.random_normal([3,3,32, 64],stddev=0.03))

#L2
L2 = tf.nn.conv2d(MP1, W2, strides=[1,1,1,1], padding='SAME')
L2_img = tf.nn.relu(L2)

#MP2
MP2 = tf.nn.max_pool(L2_img, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
MP2 = tf.reshape(MP2, [-1, 7 * 7 * 64])
# 행렬값 사이즈  none by 7*7*64

W3=tf.Variable(tf.random_normal([7*7*64,10]))
b = tf.Variable(tf.random_normal([10]))

W0722 14:27:13.084015  6056 deprecation.py:323] From <ipython-input-2-e95c973b5acc>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0722 14:27:13.085012  6056 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0722 14:27:13.086009  6056 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:


Extracting MNIST_data/train-images-idx3-ubyte.gz


W0722 14:27:13.318701  6056 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0722 14:27:13.323689  6056 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0722 14:27:13.374306  6056 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a fu

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
hypothesis = tf.matmul(MP2,W3)+b
# 결론, 1 by 10 나옴/ none 은 다 고려된 상태.

learning_rate = 0.001
# 나온 결과를 logit에 넣고 코스트 구한다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
#soft_max 는 확률값. 다 합치면 1 

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Training and Evaluation
#initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

training_epochs = 15
batch_size = 100

for epoch in range(training_epochs):
    average_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _, = sess.run ([cost, optimizer], feed_dict=feed_dict)
        average_cost += c /total_batch
    print('Epoch:', "%04d" % (epoch + 1), 'cost =', '{:.9f}'.format(average_cost))
                
print('Learning Finished!')
                
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

W0722 14:27:20.169393  6056 deprecation.py:323] From <ipython-input-3-5dc4783e915a>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch: 0001 cost = 0.275345894
Epoch: 0002 cost = 0.080431469
Epoch: 0003 cost = 0.058817459
Epoch: 0004 cost = 0.044629121
Epoch: 0005 cost = 0.037268908
Epoch: 0006 cost = 0.031892917
Epoch: 0007 cost = 0.028302175
Epoch: 0008 cost = 0.023647457
Epoch: 0009 cost = 0.020041391
Epoch: 0010 cost = 0.017056962
Epoch: 0011 cost = 0.015833107
Epoch: 0012 cost = 0.013256795
Epoch: 0013 cost = 0.012894203
Epoch: 0014 cost = 0.009607322
Epoch: 0015 cost = 0.007704942
Learning Finished!
Accuracy: 0.9873


In [8]:

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])


W1 = tf.Variable(tf.random_normal([3,3,1, 32],stddev=0.03))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1_img = tf.nn.relu(L1)
MP1 = tf.nn.max_pool(L1_img, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

print(MP1.shape)

W2 = tf.Variable(tf.random_normal([3,3,32, 64],stddev=0.03))
L2 = tf.nn.conv2d(MP1, W2, strides=[1,1,1,1], padding='SAME')
L2_img = tf.nn.relu(L2)
MP2 = tf.nn.max_pool(L2_img, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

print(MP2.shape)

MP2 = tf.reshape(MP2, [-1, 7 * 7 * 64])
print(MP2.shape)


W3=tf.Variable(tf.random_normal([7*7*64,10]))
b = tf.Variable(tf.random_normal([10]))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 3136)
